# Model Deployment

## Saving and loading the model

- Saving themodel to pickle
- Loading the model from pickle
- Turning our notebook into a Python script

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv('Telco-Customer-Churn.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [3]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [4]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod'
]

In [10]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=10000)
    model.fit(X_train, y_train)
    return dv, model

In [6]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [7]:
C = 1.0
n_splits = 5

In [11]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.842 +- 0.007


In [12]:
scores

[np.float64(0.8443689114615632),
 np.float64(0.8450763971659382),
 np.float64(0.8335661248244031),
 np.float64(0.8347329251135128),
 np.float64(0.8518166985660947)]

In [13]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values
acu = roc_auc_score(y_test, y_pred)
auc

np.float64(0.8518166985660947)

Save the model

In [14]:
import pickle # built in library

In [15]:
output_file = f'model_C={C}.bin'
output_file

'model_C=1.0.bin'

In [16]:
# Create a file
# f_out = open(output_file, 'wb') # We are going to write (w) and its a binary file (b)
# pickle.dump((dv, model), f_out) # We dump the DictVectorizer and the model to the file (With picke we can dump python objects)
# f_out.close()

# Same stuff
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

Load the model (We restarted the kernel)

In [1]:
import pickle

In [2]:
model_file = 'model_C=1.0.bin'

In [3]:
with open(model_file, 'rb') as f_in:
    dv, model = pickle.load(f_in) # To load this file we need to have sklearn installed

In [4]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=10000))

In [5]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'mutiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'payperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}


In [7]:
X = dv.transform([customer])

In [8]:
model.predict_proba(X)[0, 1]

np.float64(0.5828686127263695)

We exported the notebook to a python script, then clean and divide it in two files train.py and predict.py

## 5.3 Web services: introduction to Flask

- Installing flask

```bash
pip install flask
```

- Writing a simple ping/pong app

ping.py

```python
from flask import Flask

app = Flask('ping')

@app.route('/ping', methods=['GET'])
def ping():
    return 'PONG'

if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=9696)

```

- Querying it with `curl` and browser

```bash
curl http://0.0.0.0:9696/ping
curl http://localhost:9696/ping
```


# 5.4 Serving the Churn Model with Flask

* We create a predict service 

```python
import pickle

from flask import Flask, request, jsonify

model_file = 'model_C=1.0.bin'

with open(model_file, 'rb') as f_in:
    dv, model = pickle.load(f_in) 

app = Flask('churn')

@app.route('/predict', methods=['POST'])
def predict():
    customer = request.get_json()

    X = dv.transform([customer])
    y_pred = model.predict_proba(X)[0, 1]
    churn = y_pred >= 0.5

    result = {
        'churn_probability': float(y_pred),
        'churn': bool(churn)
    }
    
    return jsonify(result)

if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0', port=9696)
```

* And we send a post request to the service:

In [9]:
import requests

In [10]:
url = "http://localhost:9696/predict"

In [11]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'mutiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'payperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [20]:
response = requests.post(url, json=customer).json()

In [21]:
if response['churn'] == True:
    print('send promo email to %s' % ('xyz-123'))

send promo email to xyz-123


We need a WSGI server to deploy flask apps. We can use Gunicorn:

1. Install gunicorn

```bash
pip install gunicorn
```

2. We run our service with gunicorn

```bash
gunicorn --bind 0.0.0.0:9696 predict:app
```